# Day 8 - 

https://adventofcode.com/2022/day/8

In [219]:
from pathlib import Path
import ipytest

ipytest.autoconfig(addopts=["-v"])

INPUTS = Path('input.txt').read_text().strip().split('\n')

In [220]:
INPUTS[1]

'221002103112233201234202003421134043024153515443332415451142151141323114321301434333033210022221022'

The rules here state that a tree is visible if all trees from it to any edge of the map are shorter. Reversing this, that necessarily means that as we traverse from the edge towards the other end, *every* tree that is taller than all *previous* trees seen in that row is considered visible.

Given the second input row (everything in that first row will be visible, too):

***0*** ***1*** 111 ***2*** 220220 ***3*** 232333322313 ***4*** 30141411311401124212114133 ***5*** 5234334024414242133303312210313023000021220102

- Starting from the left, the `0` is visible, as that's the first tree.
- The following `1` is visible, as that's taller than the tallest tree to its left (the `0`); so this one is visible.
- The following `1`s are *not* visible: the first `1` is as tall as them.
- We keep scanning until we find the next tallest tree, a `2`.
- Following that, the next tallest, a `3`, is visible.
- Same for the next tallest, a `4`; and again for `5`.

So there are 6 trees visible *from the left side* in this line. We can reverse that same process to get trees visible *from the right side*, as well.

When we locate each of these instances, we assign coordinates for the point in the map to a set, which neatly excludes duplicates. The final result will the length of that set.

Counting left and right is simple: counting up and down is also relatively simple, but to avoid continuously accessing items in the list, it's best to iterate directly on the lists and the strings they contain. To do that, we simply transpose the original inputs with something similar to `list(zip(*inputs))` (the actual method we'll use runs a list comprehension and `"".join()` on those resulting tuples of 1-char strings).

I also wanted to avoid `range(len())` here, favoring `enumerate()` for performance and convenience of accessing sequence values. Doing that in reverse requires actually reversing the original sequence, so `enumerate(row[::-1])` was used. When this is done, though, the index returned still starts from `0`: to get the real index of the original item (to correctly identify unique coordinates), we need to do `len(row) - 1 - index)`.

Accounting for points in the transposed map means simply reversing those points when assigning them to the set. That is, `(x, y)` in the original equates to `(y, x)` in the transposed map.

Finally, in actual implementation, we just use some simple nested `for` loops. We can avoid *some* duplication, however, by counting both left and right in sequence inside the same outer loop; and the same for up and down. Like so:

```py
for row in inputs:
    for item in row:
        ...
    for item in reversed_row:
        ...

transposed = []
for col in transposed:
    for item in col:
        ...
    for item in reversed_col:
        ...
```

In [221]:
def get_visible(inputs):
    visible = set()
    for i, row in enumerate(inputs):
        # from the left
        highest_from_left = -1
        for j, val in enumerate(row):
            if int(val) > highest_from_left:
                highest_from_left = int(val)
                visible.add((i, j))

        # from the right
        row_len = len(row)
        highest_from_right = -1
        for j, val in enumerate(row[::-1]):
            actual_j = row_len - 1 - j
            if int(val) > highest_from_right:
                highest_from_right = int(val)
                visible.add((i, actual_j))

    transposed = ["".join(x) for x in zip(*inputs)]
    for i, row in enumerate(transposed):
        highest_from_top = -1
        # from the top
        for j, val in enumerate(row):
            if int(val) > highest_from_top:
                highest_from_top = int(val)
                visible.add((j, i))

        # from the bottom
        row_len = len(row)
        highest_from_bottom = -1
        for j, val in enumerate(row[::-1]):
            actual_j = row_len - 1 - j
            if int(val) > highest_from_bottom:
                highest_from_bottom = int(val)
                visible.add((actual_j, i))

    return len(visible)


In [222]:
%%ipytest

def test_get_visible():
    inputs = [
        "30373",
        "25512",
        "65332",
        "33549",
        "35390",
    ]
    result = get_visible(inputs)
    assert result == 21


======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day08
collecting ... collected 1 item

t_d69bfed3f39c475eb6985a1c7ff68a00.py::test_get_visible PASSED                               [100%]

======================================== 1 passed in 0.01s =========================================


In [223]:
result = get_visible(INPUTS)
print(f"{result=}")

result=1787


## Part 2

Different approach here. We'll need to create a new map that sets up all the "scenic scores" for each tree, initialized at 1. Then we traverse the inputs as before, calculating directional scenic scores as we go for each particular point, and doing `score *= directional` (each directional could be `1+`, so default of 1 * these values should be accurate).

To calculate a single directional score, let's consider our second row from `INPUTS` again:

221002103112233201234202003421134043024153515443332415451142151141323114321301434333033210022221022

For any given point, we can determine all trees in the sight line of that direction using a slice. Consider the first `2` to be found to the right of the first `3`:

22100210311 ***2*** 233201234202003421134043024153515443332415451142151141323114321301434333033210022221022

The slice to its left is essentially `row[:idx]`, where `idx` is the index of that `2`.

We can pass that to a helper function that determines the sight score, given:
- The slice we provide should be aligned left-right, where index `0` is adjacent to our target.
- Pass the value of the target, in this case `2`.

The function should then find the next value from the left that is `>=` to the target. If located, the index of that position + 1 is our result, similar to the length of a list sliced up to that position.

In [224]:
from typing import Sequence

def get_directional_scenic_score(tree: int, nearby: Sequence[int]) -> int:
    for idx, val in enumerate(nearby, start=1):
        if val >= tree:
            # Found one equal or taller
            return idx
    # Found nothing taller: we can see clear to the edge
    return len(nearby)

First up, it would make things easier to convert our inputs to ints, finally.

In [225]:
trees = [list(map(int, x)) for x in INPUTS]

In [226]:
def get_scenic_scores(trees: list[list[int]]) -> list[list[int]]:
    scores = [[1 for _ in x] for x in trees]

    for i, row in enumerate(trees):
        # Traverse the row, but don't bother with the edge.
        for j, tree in enumerate(row[1:], start=1):
            # The nearby trees extend from the beginning of the row up to our current tree.
            # We also need to reverse that set to align them correctly before getting a score.
            nearby = row[:j][::-1]
            scores[i][j] *= get_directional_scenic_score(tree, nearby)

        row_len = len(row)
        # Now traverse the row backwards, and again skip the first item
        for j, tree in enumerate(row[::-1][1:], start=1):
            actual_j = row_len - 1 - j
            # The original row is unchanged (not reversed),
            # so this time go from actual_j to the end (to the right of the target)
            # We also need to add 1 to exclude the target from that slice
            nearby = row[actual_j + 1 :]
            scores[i][actual_j] *= get_directional_scenic_score(tree, nearby)

    # Now we run the same operations on the transposed set.
    # Remember as before, reverse the coordinates we used when assigning scores
    transposed = list(zip(*trees))
    for i, row in enumerate(transposed):
        for j, tree in enumerate(row[1:], start=1):
            nearby = row[:j][::-1]
            scores[j][i] *= get_directional_scenic_score(tree, nearby)

        row_len = len(row)
        for j, tree in enumerate(row[::-1][1:], start=1):
            actual_j = row_len - 1 - j
            nearby = row[actual_j + 1 :]
            scores[actual_j][i] *= get_directional_scenic_score(tree, nearby)

    return scores


In [227]:
%%ipytest

def test_get_scores():
    inputs = [
        [3, 0, 3, 7, 3],
        [2, 5, 5, 1, 2],
        [6, 5, 3, 3, 2],
        [3, 3, 5, 4, 9],
        [3, 5, 3, 9, 0],
    ]
    result = get_scenic_scores(inputs)
    # Known positions from the thing
    assert result[1][2] == 4
    assert result[3][2] == 8

======================================= test session starts ========================================
platform darwin -- Python 3.10.7, pytest-7.2.0, pluggy-1.0.0 -- /Users/garice/Library/Caches/pypoetry/virtualenvs/griceturrble-advent-of-code-8jQN35Cx-py3.10/bin/python
cachedir: .pytest_cache
rootdir: /Users/garice/dev/gits/personal/advent-of-code/2022/day08
collecting ... collected 1 item

t_d69bfed3f39c475eb6985a1c7ff68a00.py::test_get_scores PASSED                                [100%]

======================================== 1 passed in 0.01s =========================================


Now we can get our scenic scores for all the trees from inputs.

The ones at the edges - first and last row, first and last col in each row - are to be ignored, as we only care about trees that have sight lines on all sides.

So, gather all the scores from our matrix except those excluded ones, and find the largest one. Easy enough.

In [228]:
scores = get_scenic_scores(trees)[1:-1]
all_scores = [x for y in scores for x in y[1:-1]]
result = max(all_scores)
print(f"{result=}")


result=440640
